In [7]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

T = 1 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [8]:
fes = H1(mesh, order=1, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 1 # the wind speed
eps = 0.001 # the diffusion coefficient

phi_expr = (1 - exp((x - 1)/eps)) / (1 - exp(-1/eps))
uexact = phi_expr * exp(-y)  # y = Zeit

# Ableitungen für f(x,t)
phi = (1 - exp((x - 1)/eps)) / (1 - exp(-1/eps))
phi_x = (1/eps) * exp((x - 1)/eps) / (1 - exp(-1/eps))
phi_xx = (1/eps**2) * exp((x - 1)/eps) / (1 - exp(-1/eps))

# Quelle f(x,t)
f_expr = (-phi + (w - 1)/eps * exp((x - 1)/eps) / (1 - exp(-1/eps))) * exp(-y)

ux = CoefficientFunction((grad(u)[0]))
B = CoefficientFunction((w,1))
a = BilinearForm(fes, symmetric=False)
a += eps*ux*grad(v)[0]*dx 
a += (B*grad(u))*v*dx
a.Assemble()

f = LinearForm(fes)
f += f_expr*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(phi_expr,  BND)  # initial condition
#gfu.Set(1,  definedon=mesh.Boundaries("right"))  # Dirichlet BC
#gfu.Set(0, definedon=mesh.Boundaries("left"))  # Dirichlet BC

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)

l2error = sqrt(Integrate((gfu-(uexact))**2, mesh))
print(l2error)
Draw(uexact, mesh, "exact")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.296687727765742


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [9]:
order =3
fes = H1(mesh, order=order, dirichlet="bottom|right|left")
u,v = fes.TnT()
n = specialcf.normal(2)
w = 1 # the wind speed
eps = 0.01 # the diffusion coefficient

h = specialcf.mesh_size
jac = specialcf.JacobianMatrix(mesh.dim)
M = CF( [[2/sqrt(3), 1/sqrt(3)], [1/sqrt(3), 2/sqrt(3)]] )
expr = jac.trans * M * jac
Cinv = 36*order**2
ws = CF((w,1))
tau =  InnerProduct(ws,expr *ws)
gamma_T = 1/sqrt(tau + (Cinv*eps/h**2)**2)

def gradx(u):
    return grad(u)[0]
def dt(u):
    return grad(u)[1]

uxx = u.Operator("hesse")[0,0]

a = BilinearForm(fes, symmetric=False)
a += eps*gradx(u)*gradx(v)*dx + (ws*grad(u))*v*dx
a += gamma_T * (-eps*uxx + ws * grad(u))*(ws* grad(v)) * dx
a.Assemble()

f = LinearForm(fes)
f += f_expr*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(phi_expr,  BND)  # initial condition
#gfu.Set(1,  definedon=mesh.Boundaries("right"))  # Dirichlet BC
#gfu.Set(0, definedon=mesh.Boundaries("left"))  # Dirichlet BC
Draw(gfu)
res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)


l2error = sqrt(Integrate((gfu-(uexact))**2, mesh))
print(l2error)
Draw(uexact, mesh, "exact")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.22316990097361358


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene